In [6]:
import cv2
import numpy as np 
import os 
import glob 
import matplotlib.pyplot as plt

In [7]:

#checkerboard used size is 8,8 with 2m spacing but using a 7,7 input was the only way the calibration worked

CHECKERBOARD = (7,7)

subpix_criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER,30,0.1)

#set calibration flags for fisheye camera
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC + cv2.fisheye.CALIB_CHECK_COND + cv2.fisheye.CALIB_FIX_SKEW

objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)

_img_shape = None
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('images/*.jpg')
print(images)


['images\\image_0.jpg', 'images\\image_1.jpg', 'images\\image_10.jpg', 'images\\image_11.jpg', 'images\\image_12.jpg', 'images\\image_13.jpg', 'images\\image_14.jpg', 'images\\image_15.jpg', 'images\\image_16.jpg', 'images\\image_17.jpg', 'images\\image_18.jpg', 'images\\image_19.jpg', 'images\\image_2.jpg', 'images\\image_20.jpg', 'images\\image_21.jpg', 'images\\image_22.jpg', 'images\\image_23.jpg', 'images\\image_24.jpg', 'images\\image_25.jpg', 'images\\image_26.jpg', 'images\\image_27.jpg', 'images\\image_28.jpg', 'images\\image_29.jpg', 'images\\image_3.jpg', 'images\\image_30.jpg', 'images\\image_31.jpg', 'images\\image_32.jpg', 'images\\image_33.jpg', 'images\\image_34.jpg', 'images\\image_35.jpg', 'images\\image_36.jpg', 'images\\image_37.jpg', 'images\\image_38.jpg', 'images\\image_39.jpg', 'images\\image_4.jpg', 'images\\image_40.jpg', 'images\\image_5.jpg', 'images\\image_6.jpg', 'images\\image_7.jpg', 'images\\image_8.jpg', 'images\\image_9.jpg']


In [5]:
#### this is cell is just for tests 
# CHECKERBOARD = (7,7)   #7,7 works for gray_3 and 6,9 works for 1 and 2
# gray_1 = cv2.imread("cc.jpg")
# gray_2 = cv2.imread("calib.jpg")
# gray_3 = cv2.imread("images/image_0.jpg")
# gray_1 = cv2.cvtColor(gray_1,cv2.COLOR_BGR2GRAY)
# gray_2 = cv2.cvtColor(gray_2,cv2.COLOR_BGR2GRAY)
# gray_3 = cv2.cvtColor(gray_3,cv2.COLOR_BGR2GRAY)

# now_img = gray_3
# ret, corners = cv2.findChessboardCorners(now_img, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
# print(ret)
# plt.imshow(now_img)

In [8]:
rt = 0
rf = 0
for fname in images:
    img = cv2.imread(fname)
    if _img_shape == None:
        _img_shape = img.shape[:2]
    else:
        assert _img_shape == img.shape[:2], "All images must share the same size."
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#     plt.imshow(gray)
    # cv2.waitKey(0)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
#     print(ret)
    # If found, add object points, image points (after refining them)
    if ret == True:
        rt+=1
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
        imgpoints.append(corners)
    else:
        rf +=1
        print(fname)
        
print("done")
# print("number of trues ={},number of falses ={}".format(rt,rf))

images\image_11.jpg
images\image_13.jpg
images\image_14.jpg
images\image_16.jpg
images\image_18.jpg
images\image_21.jpg
images\image_22.jpg
images\image_23.jpg
images\image_24.jpg
images\image_25.jpg
images\image_28.jpg
images\image_29.jpg
images\image_30.jpg
images\image_31.jpg
images\image_32.jpg
images\image_33.jpg
images\image_34.jpg
images\image_35.jpg
images\image_37.jpg
images\image_5.jpg
images\image_6.jpg
done


In [4]:
N_OK = len(objpoints)
K = np.zeros((3, 3))
D = np.zeros((4, 1))
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
rms, _, _, _, _ = \
    cv2.fisheye.calibrate(
        objpoints,
        imgpoints,
        gray.shape[::-1],
        K,
        D,
        rvecs,
        tvecs,
        calibration_flags,
        (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
    )
print("Found " + str(N_OK) + " images for calibration")
print("number of useful ={},number of false images ={}".format(rt,rf))
print("DIM=" + str(_img_shape[::-1]))
print("K=np.array(" + str(K.tolist()) + ")")
print("D=np.array(" + str(D.tolist()) + ")")

Found 20 images for calibration
number of useful =20,number of false images =21
DIM=(640, 720)
K=np.array([[159.99575739328887, 0.0, 319.5831640048945], [0.0, 160.02685012507118, 359.32876776861923], [0.0, 0.0, 1.0]])
D=np.array([[0.0771116403943704], [0.024655141427782377], [-0.014563702458641147], [0.004491653395838882]])
